In [ ]:
%pylab inline

In [ ]:
import pandas as pd
import json
from datetime import datetime
import numpy as np
from fbprophet import Prophet
import pickle
import math

# Read Bitstamp Data

First, let's read in the data from Kaggle.

In [ ]:
df = pd.read_csv('bitstampUSD_1-min_data_2012-01-01_to_2018-03-27.csv')

In [ ]:
df['ds'] = df['Timestamp'].apply(lambda x: datetime.fromtimestamp(x))
df = df.set_index(df['ds'])

# Train Model Using Prophet

Next, let's take the log of weighted price and train our model using that feature.

In [ ]:
df['y'] = np.log(df['Weighted_Price'])

In [ ]:
df_subset = df[df['ds'] >= pd.Timestamp('2016-01-01')]

In [ ]:
m = Prophet()

In [ ]:
%time m.fit(df_subset[['ds','y']]);

Only run the below if you want to save a **NEW** model

In [ ]:
with open('btc_model.pickle', 'wb') as pickle_file:
    pickle.dump(m, pickle_file)
    pickle_file.close()

Use this if you've already trained the model

In [ ]:
with open('btc_model.pickle', 'rb') as pickle_file:
    m = pickle.Unpickler(pickle_file)
    m = m.load()

# Generate Prediction

Now, let's predict out prices using the model we just trained.

In [ ]:
future = m.make_future_dataframe(periods=150000, freq='1min', include_history=False)
future.tail()

In [ ]:
%time fcst = m.predict(future)

We can use our forecast (`fcst`) to plot the prediction we just made:

In [ ]:
m.plot(fcst);

Now let's convert our `log(price)` back to price and calculate whether the minute is a buy or sell minute

In [ ]:
fcst['yhat_exp'] = fcst['yhat'].apply(lambda x: math.exp(x))

In [ ]:
output = fcst[fcst['ds'] > '2018-01-01'][['ds','yhat_exp']]

In [ ]:
output['delta'] = output.yhat_exp.diff()

In [ ]:
output = output[output['ds'] > '2017-12-25']

In [ ]:
def choose_side(x):
    if x > 0:
        return 'buy'
    else: 
        return 'sell'

In [ ]:
output['side'] = output['delta'].apply(lambda x: choose_side(x))

We can count the number of buy vs. sell minutes with:

In [ ]:
output.groupby('side')['ds'].count()

# Store Output

Save file to CSV

In [ ]:
output[['ds','side']].to_csv('2018_full_trade_forecast.csv', index=False)

# Analyze Output

Generate a components plot

In [ ]:
fcst_sampled = fcst[fcst['ds'] > '2018-01-01 00:00:00']

In [ ]:
a = m.plot_components(fcst_sampled)

You can save the components plot to a file if you'd like

In [ ]:
a.savefig('components_plot.png', dpi=200)